In [47]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from salishsea_tools import evaltools as et, viz_tools
import os
import scipy.interpolate as sinterp
import cmocean
import netCDF4 as nc
import xarray as xr

In [48]:
path_run = PATH= '/home/jvalenti/scratch/run_SHEM/SHEM18' #tuning/pred_flag' 
SalishSeadf = pd.read_csv('/home/jvalenti/MOAD/analysis-jose/notebooks/SHEM/eval/SalishSea_MesozoopModelvsObsMatches_2015to2019_Eval202111.csv', index_col=0,header=0,parse_dates= ['Date','dtUTC'])

In [49]:
# date_original = SalishSeadf['dtUTC']
# date_altered = []

# for t in date_original:
#     date_altered.append(t.replace(year=2021))


# SalishSeadf['dtUTC'] = date_altered


In [50]:
def make_filename(path_run,start,day=0, var='biol_T', res='h'):
    """Construct path prefix for local SHEM results given date object and paths dict
    """
    date  =  (start + timedelta(days=day))
    folder = date.strftime("%d%b%y").lower()
    prefix = os.path.join(path_run, f'{folder}/')
    fname = []
    try:
        for file in os.listdir(prefix):
            if (var in file) and ('_1'+res) in file:
                fname.append(file)
        if len(fname)>1:
            print('more than one file found') 
        try:
            path = os.path.join(prefix, fname[0])
            return path
        except IndexError:
            print('no file available')
    except FileNotFoundError:
        print('no file available')
    

In [51]:
M_path = '/home/jvalenti/MOAD/'
mask = xr.open_dataset(M_path+'grid2/mesh_mask202108_TDV.nc')

In [52]:
SalishSeadf = SalishSeadf[SalishSeadf.dtUTC>datetime(2018,1,1)]
SalishSeadf = SalishSeadf[SalishSeadf.dtUTC<datetime(2018,8,31)]

In [33]:
# example = SalishSeadf.iloc[0]
# start = example.dtUTC.to_pydatetime()    
# mmolNm2 = (xr.open_dataset(make_filename(path_run,start)).mesozooplankton[0,example.k_upper:example.k_lower+1,example.j,example.i]
#            *mask.e3t_0[0,example.k_upper:example.k_lower+1,example.j,example.i]).sum()

In [53]:
total_mine = np.zeros(len(SalishSeadf))
micro_mine = np.zeros(len(SalishSeadf))
meso_mine = np.zeros(len(SalishSeadf))

for n in range(len(SalishSeadf)):
    example = SalishSeadf.iloc[n]
    start = example.dtUTC.to_pydatetime()    
    jj = example.j
    ii = example.i
    k_u = example.k_upper
    k_l = example.k_lower
    e3tt = np.array(mask.e3t_0[0,k_u:k_l+1,jj,ii])
    masktt = np.array(mask.tmask[0,k_u:k_l+1,jj,ii])
    res = 'd'
    var = 'biol_T'
    try:
        meso_mine[n] = sum(np.array(xr.open_dataset(make_filename(path_run,start,var=var,res=res)).mesozooplankton[0,k_u:k_l+1,jj,ii])*e3tt*masktt)/sum(e3tt*masktt)
        micro_mine[n] = sum(np.array(xr.open_dataset(make_filename(path_run,start,var=var,res=res)).microzooplankton[0,k_u:k_l+1,jj,ii])*e3tt*masktt)/sum(e3tt*masktt)
        total_mine[n] = float(meso_mine[n]) + float(micro_mine[n])
    except ValueError or FileNotFoundError:
        total_mine[n]=np.nan
        micro_mine[n]=np.nan
        meso_mine[n]=np.nan

In [54]:
SalishSeadf['SHEM_mesozooplankton']=meso_mine
SalishSeadf['SHEM_microzooplankton']=micro_mine
SalishSeadf['SHEM_total']=total_mine

In [55]:
SalishSeadf['SHEM_microzooplanktonDI']=(SalishSeadf['SHEM_mesozooplankton'])*SalishSeadf['Z_lower']
SalishSeadf['SHEM_mesozooplanktonDI']=(SalishSeadf['SHEM_mesozooplankton'])*SalishSeadf['Z_lower']
SalishSeadf['SHEM_totalDI']=(SalishSeadf['SHEM_total'])*SalishSeadf['Z_lower']

In [56]:
# define log transform function with slight shift to accommodate zero values
def logt(x):
  return np.log10(x+.001)
# define inverse log transform with same shift
def logt_inv(y):
    return 10**y-.001

In [57]:
#Convert SHEMel values to mg C m-3 by muliplying value * C:N of 5.7 * molecular weight of C
SalishSeadf['L10SHEM_mesozooplankton']=logt(SalishSeadf['SHEM_mesozooplankton']*5.7*12)
SalishSeadf['L10SHEM_mesozooplanktonDI']=logt(SalishSeadf['SHEM_mesozooplanktonDI']*5.7*12/1000)
SalishSeadf['logt_invL10SHEM_mesozooplanktonDI']=logt_inv(SalishSeadf['L10SHEM_mesozooplanktonDI'])

#Convert SHEMel values to mg C m-3 by muliplying value * C:N of 5.7 * molecular weight of C
SalishSeadf['L10SHEM_microooplankton']=logt(SalishSeadf['SHEM_microzooplankton']*5.7*12)
SalishSeadf['L10SHEM_microzooplanktonDI']=logt(SalishSeadf['SHEM_microzooplanktonDI']*5.7*12/1000)
SalishSeadf['logt_invL10SHEM_microzooplanktonDI']=logt_inv(SalishSeadf['L10SHEM_microzooplanktonDI'])

#Convert SHEMel values to mg C m-3 by muliplying value * C:N of 5.7 * molecular weight of C
SalishSeadf['L10SHEM_total']=logt(SalishSeadf['SHEM_total']*5.7*12)
SalishSeadf['L10SHEM_totalDI']=logt(SalishSeadf['SHEM_totalDI']*5.7*12/1000)
SalishSeadf['logt_invL10SHEM_totalDI']=logt_inv(SalishSeadf['L10SHEM_totalDI'])

In [58]:
SalishSeadf

,Unnamed: 0,Key,region_name,Lat,Lon,Date,Z_lower,Z_upper,dtUTC,YD,...,SHEM_totalDI,L10SHEM_mesozooplankton,L10SHEM_mesozooplanktonDI,logt_invL10SHEM_mesozooplanktonDI,L10SHEM_microooplankton,L10SHEM_microzooplanktonDI,logt_invL10SHEM_microzooplanktonDI,L10SHEM_total,L10SHEM_totalDI,logt_invL10SHEM_totalDI
1462,0,010218ELIV1151,Bellingham Bay,48.63795,-122.5694,2018-01-02 00:00:00,110.0,0,2018-01-02 19:51:00,2,...,62.823033,1.493544,0.535050,3.427069,0.898195,0.535050,3.427069,1.591793,0.633276,4.297095
1463,84,030118MUKV1011,Whidbey Basin,47.97166,-122.3222,2018-01-03 00:00:00,196.0,0,2018-01-03 18:11:00,3,...,68.492491,1.242103,0.534461,3.422429,0.809028,0.534461,3.422429,1.378461,0.670792,4.684886
1464,1,010818SKETV1058,South Sound,47.15243,-122.6586,2018-01-08 00:00:00,122.0,0,2018-01-08 18:58:00,8,...,75.148962,1.468912,0.555378,3.591345,1.103682,0.555378,3.591345,1.624630,0.711064,5.140189
1465,1344,UBC9999099001901,Central Strait of Georgia,49.25000,-123.6670,1/15/2018,375.0,0,2018-01-15 20:00:00,15,...,58.393258,0.858988,0.433119,2.709936,0.534716,0.433119,2.709936,1.027428,0.601528,3.994099
1466,11,011618KSBP01V0909,Central Basin,47.74396,-122.4282,2018-01-16 00:00:00,200.0,0,2018-01-16 17:09:00,16,...,97.826628,1.338440,0.639549,4.359632,1.066682,0.639549,4.359632,1.524496,0.825578,6.691341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701,1339,UBC9999099001201,Tidal Mixed,48.63000,-123.2430,8/27/2018,138.0,0,2018-08-27 17:35:00,239,...,198.455931,1.648129,0.788069,6.137601,1.731514,0.788069,6.137601,1.992846,1.132752,13.574386
1702,830,082718ELIV1037,Bellingham Bay,48.63795,-122.5694,2018-08-27 00:00:00,109.0,0,2018-08-27 17:37:00,239,...,208.847206,1.778540,0.816025,6.545741,1.851287,0.816025,6.545741,2.117462,1.154915,14.285149
1703,1341,UBC9999099001401,Juan de Fuca,48.30000,-124.0000,8/27/2018,104.0,0,2018-08-28 01:45:00,240,...,118.644019,1.582170,0.599302,3.973676,1.600143,0.599302,3.973676,1.892274,0.909355,8.115251
1704,845,083018MUKV1006,Whidbey Basin,47.97166,-122.3222,2018-08-30 00:00:00,192.0,0,2018-08-30 17:06:00,242,...,319.568369,1.699485,0.982823,9.611194,1.804745,0.982823,9.611194,2.056322,1.339640,21.858476


In [59]:
SalishSeadf.to_csv('evaldata_shem_2.csv')

In [60]:
#print(monthlymean['Total'].max())
#print(monthlymean['mod_mesozooplankton'].max()*5.7*12)